In [1]:
#DASHBOARD
import plotly.express as px
import plotly.graph_objs as go
import dash 
from dash import dcc, ctx
import dash_bootstrap_components as dbc
from dash import html
from jupyter_dash import JupyterDash
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate


import warnings
warnings.filterwarnings("ignore")

In [2]:
#PROCESS
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import mapbox
from shapely.geometry import Point, Polygon, MultiPoint, GeometryCollection
from sklearn.cluster import DBSCAN
from alphashape import alphashape
import os


In [3]:
app = JupyterDash(external_stylesheets=[dbc.themes.DARKLY])

mapbox_token = 'pk.eyJ1IjoiYmVybW9kYTA0IiwiYSI6ImNsZjBnbGxodjAxeHgzcm81eTRlazF5eDEifQ.0CPIhwqhMinleOCQ4sRHlQ'
px.set_mapbox_access_token(mapbox_token)

In [4]:
#DATASET

# Set the directory path
dir_path = "C:/Users/Acer/Documents/SchoolHard/Thesis/Code/fin_csv/UniKrig"

# Get all file names inside the directory
file_names = os.listdir(dir_path)

# Concatenate the file names with the directory path
file_paths = [os.path.join(dir_path, file_name) for file_name in file_names]

dummy_df = pd.read_csv(file_paths[0])

dummy_gdf = gpd.GeoDataFrame(
    dummy_df, 
    geometry=gpd.points_from_xy(dummy_df.lon, dummy_df.lat)
)

In [5]:
#global variables
fig = None
cluster_gdf = None
bar_fig = None
all_gdf = None

In [6]:
#MULTI-CRITERIA DECISION MAKING

#PFZ threshold
def sst_threshold(sst):
    if sst < 24 or sst > 30:
        return 1.0
    elif sst > 29 and sst <= 30:
        return 2.0
    elif sst > 24 and sst <= 29:
        return 3.0
    else:
        return 0
    
#CHL-a threshold
def chla_threshold(chl_a):
    if chl_a < .2:
        return 1.0
    elif chl_a >= .2 and chl_a <= .5:
        return 2.0
    elif chl_a > .5:
        return 3.0
    else:
        return 0

#MCDS
def MCDS (df) : 
    tf_weights = {'low': 1, 'moderate': 2, 'high': 3}
    pfz_df = df.loc[:, ['lat', 'lon']]

    pfz_df['sst_thresh'] = df['mean_sst'].apply(sst_threshold)
    pfz_df['chla_thresh'] = df['mean_chla'].apply(chla_threshold)
    pfz_df['tf_thresh'] = df['thermal_mask'].map(tf_weights)

    #weights for the variable for PFZ criteria
    w_sst = 0.333  
    w_chla = 0.333 
    w_tf = 0.333

    pfz_df['PFZ_score'] = w_sst*pfz_df['sst_thresh'] + w_chla*pfz_df['chla_thresh'] + w_tf*pfz_df['tf_thresh']
    pfz_df['pfz'] = np.where(pfz_df['PFZ_score'] >= 2.3, 'High', np.where(pfz_df['PFZ_score'] > 1.6, 'Moderate', 'Low'))

    return pfz_df['PFZ_score'], pfz_df['pfz']

In [7]:
#DAVAO GULF SECTORS
dg_shape = gpd.read_file('C:/Users/Acer/Documents/SchoolHard/Thesis/Code/shp_files/dgp_divided_v3.shp')

converted_polygon = dg_shape['geometry'].buffer(0.0).simplify(0.0001, preserve_topology=False)
converted_area = converted_polygon.area * (111319.9 ** 2)
#area_m2 = dg_shape.area
area_km2 = converted_area / 1000000
dg_shape['area_km2'] = area_km2

dg_shape_final = dg_shape[dg_shape['area_km2'] != 0]
dg_shape_final['color_representation'] = 'Sea'

In [8]:
cluster_group_area = dg_shape_final[['POLY_ID', 'area_km2']]
cluster_group_area['High_area_km2'] = 0
cluster_group_area['Moderate_area_km2'] = 0
cluster_group_area['Low_area_km2'] = 0

def new_cga():
    global cluster_group_area
    
    cluster_group_area = dg_shape_final[['POLY_ID', 'area_km2']]
    cluster_group_area['High_area_km2'] = 0
    cluster_group_area['Moderate_area_km2'] = 0
    cluster_group_area['Low_area_km2'] = 0

In [9]:
# function that generates a scatter mapbox figure and adds a choropleth overlay
def map_figure(mdf, color_drop):
    
    global fig  # declaring fig as a global variable so it can be accessed outside of the function
    
    # initialize color and color mode variables
    color = None
    color_mode = None
    
    # if statement to determine which color scheme to use based on the user's selection
    if color_drop == 2:
        color = 'pfz'
        color_mode = {'High': 'greenyellow', 'Moderate': 'steelblue', 'Low': 'midnightblue'}
        fig = px.scatter_mapbox(mdf,  # plot the scatter mapbox figure
                                lon=mdf['lon'],
                                lat=mdf['lat'],
                                hover_name='pfz',  # add 'pfz' as hover text
                                hover_data=['PFZ_score', 'mean_sst', 'mean_chla', 'mean_tm'],  # additional hover data
                                color=color,  # set color based on pfz values
                                color_discrete_map=color_mode,  # map color values to specific colors
                                opacity=.8)
        
    elif color_drop == 1:  # 1st option
        color = 'PFZ_score'
        color_mode = 'Viridis'
        fig = px.scatter_mapbox(mdf, lon=mdf['lon'], lat=mdf['lat'], hover_name='pfz',
                                hover_data=['PFZ_score', 'mean_sst', 'mean_chla', 'mean_tm'],
                                color=color, color_continuous_scale=color_mode, range_color = [1,2.6], opacity=.8)
    
    elif color_drop == 3:  # 3rd option
        color = 'mean_sst'
        color_mode = 'turbo'
        fig = px.scatter_mapbox(mdf, lon=mdf['lon'], lat=mdf['lat'], hover_name='pfz',
                                hover_data=['PFZ_score', 'mean_sst', 'mean_chla', 'mean_tm'],
                                color=color, color_continuous_scale=color_mode, opacity=.8)
    
    elif color_drop == 4:  # 4th option
        color = 'mean_chla'
        color_mode = 'Viridis'
        fig = px.scatter_mapbox(mdf, lon=mdf['lon'], lat=mdf['lat'], hover_name='pfz',
                                hover_data=['PFZ_score', 'mean_sst', 'mean_chla', 'mean_tm'],
                                color=color, color_continuous_scale=color_mode, opacity=.8)
        
    else:  # 5th option
        color = 'mean_tm'
        #color_mode = "YlGnBu_r"
        color_mode = "deep_r"
        fig = px.scatter_mapbox(mdf, lon=mdf['lon'], lat=mdf['lat'], hover_name='pfz',
                                hover_data=['PFZ_score', 'mean_sst', 'mean_chla', 'mean_tm'],
                                color=color, color_continuous_scale=color_mode,range_color = [0, 0.03], opacity=.8)

    # create choropleth mapbox figure using the dg_shape_final data
    chor_fig = px.choropleth_mapbox(dg_shape_final, geojson=dg_shape_final.geometry, color='color_representation',
                                    color_discrete_map={'Sea': 'midnightblue'}, locations=dg_shape_final.index,
                                    opacity=.7)

    # update the layout of the scatter mapbox figure
    fig.update_layout(
        mapbox=dict(
            center=dict(lat=6.747123, lon=125.7020),  # set the center of the map
            zoom=9,  # set the initial zoom level
            style="mapbox://styles/mapbox/dark-v10"),  # set the style of the map
        margin=dict(l=0, r=0, t=0, b=0),  # set the margin
        legend=dict(  # create legend
            bgcolor='rgba(255, 255, 255, 0.7)',
            x=0.05,
            y=0.05,
            traceorder='normal',
            font=dict(size=12),
            title_font=dict(size=14),
            orientation='h',
            yanchor='bottom',
            xanchor='left',
            bordercolor='#FFFFFF',
            borderwidth=1,
            itemsizing='trace',
            title=dict(text='Legend', side='top')
        )
    )

    # add the choropleth figure to the scatter mapbox figure
    fig.add_trace(chor_fig.data[0])


In [10]:
sector_points_dict = {}
global all_gdf

poly_dir_path = "C:/Users/Acer/Documents/SchoolHard/Thesis/Code/POLYGON_POINTS"

# Get all file names inside the directory
poly_file_names = os.listdir(poly_dir_path)

# Concatenate the file names with the directory path
poly_file_paths = [os.path.join(poly_dir_path, poly_file_name) for poly_file_name in poly_file_names]


for index in range(len(poly_file_paths)):
    
    # Load the data
    df = pd.read_csv(poly_file_paths[index])
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.lon, df.lat))
    gdf.crs = 'WGS84'
    sector_points_dict[index] = gdf
    
    if index == 0:
        all_gdf = gdf
    else:
        all_gdf = pd.concat([all_gdf, gdf])

print("nice")

nice


In [11]:
def clustering(PFZ):
    
    # create an empty list to store information about each cluster
    cluster_polygons = []

    # loop through each key-value pair in the sector_points_dict dictionary
    for key, sector_points in sector_points_dict.items():

        # extract data for the current sector
        for_cluster = sector_points_dict[key]

        # check if any of the 'pfz' values for this sector contain the PFZ value passed to the function
        if for_cluster['pfz'].str.contains(str(PFZ)).any() :

            # filter the data for this sector to only include the PFZ value passed to the function
            for_cluster = for_cluster[for_cluster['pfz'] == str(PFZ)]

            # perform DBSCAN clustering on the lon/lat coordinates in the data
            dbscan = DBSCAN(eps= .006, min_samples= 4)
            for_cluster['cluster'] = dbscan.fit_predict(for_cluster[['lon', 'lat']])

            # remove any points that were not assigned to a cluster
            for_cluster = for_cluster[for_cluster['cluster'] != -1]

            # loop through each cluster identified by the algorithm
            for cluster_id in for_cluster['cluster'].unique():

                # extract the lon/lat coordinates for this cluster
                cluster_points = for_cluster[for_cluster['cluster'] == cluster_id]
                cluster_points = cluster_points[['lon', 'lat']].values

                # calculate the concave hull polygon for this cluster
                concave_hull = alphashape(cluster_points, alpha = 100)
                if  isinstance(concave_hull, Polygon):
                    cluster_polygon = Polygon(list(concave_hull.exterior.coords))

                    # add information about this cluster to the cluster_polygons list
                    cluster_polygons.append({'cluster_id': cluster_id, 'geometry': cluster_polygon, 'polygon_id': key})

    # convert the cluster_polygons list to a GeoDataFrame
    global cluster_gdf
    cluster_gdf = gpd.GeoDataFrame(cluster_polygons, crs=dummy_gdf.crs)

    # remove the 'cluster_id' column (which is redundant with the index)
    cluster_gdf.pop('cluster_id')

    # reassign the index to be the 'cluster_id' column
    clustered_index = cluster_gdf.index
    cluster_gdf['cluster_id'] = clustered_index

    # reorder the columns in the GeoDataFrame
    cluster_gdf = cluster_gdf[['cluster_id', 'geometry', 'polygon_id']]

    # calculate the area (in km^2) of each cluster polygon and add it as a new column
    converted_polygon = cluster_gdf['geometry'].buffer(0.0).simplify(0.0001, preserve_topology=False)
    converted_area = converted_polygon.area * (111319.9 ** 2)
    area_km2 = converted_area / 1000000
    cluster_gdf['area_km2'] = area_km2

    # add a new column to represent the color of the clusters in the resulting map
    cluster_gdf['color_representation'] = str(PFZ) + ' PFZ Clusters'


In [12]:
cluster_gdf

In [13]:
def cluster_trace(pfz):
    
    # global variables fig, cluster_gdf, and cluster_group_area
    global fig
    global cluster_gdf
    global cluster_group_area
    
    # Calls the function clustering and passes pfz as an argument
    clustering(pfz)
    color = None # Initializes variable color to None
    
    
    # Assigns a color based on the pfz parameter
    if pfz == "Low":
        color = "midnightblue"
    elif pfz == "Moderate":
        color = "steelblue"
    else:
        color = "greenyellow"
    
    # Creates a choropleth map
    chor_fig = px.choropleth_mapbox(cluster_gdf, 
                        geojson=cluster_gdf.geometry, 
                        color=cluster_gdf['color_representation'],
                        color_discrete_map={str(pfz) + ' PFZ Clusters': color},
                        locations=cluster_gdf.index, 
                        opacity=.5,)

    # Adds a trace to the fig plot. The trace is the first data element of chor_fig
    fig.add_trace(
        chor_fig.data[0]
    )

    # Groups the data in cluster_gdf by polygon_id and calculates the sum of area_km2. 
    cluster_group_area[pfz + "_area_km2"] = cluster_gdf.groupby('polygon_id')['area_km2'].sum()
    cluster_group_area = cluster_group_area.fillna(0) # Any missing values are filled with 0
    

In [14]:
#MAPBOX TOKEN
mapbox_token = 'pk.eyJ1IjoiYmVybW9kYTA0IiwiYSI6ImNsZjBnbGxodjAxeHgzcm81eTRlazF5eDEifQ.0CPIhwqhMinleOCQ4sRHlQ'
px.set_mapbox_access_token(mapbox_token)

In [15]:
# App layout
app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("PFZ DASH", style={'margin-left': '35px', 
                                                 'margin-top': '90px'}),
            html.P("FISHING ZONE MAP: A web application for finding the best fishing sites.", style={'margin-left': '35px'}),
            html.Label("Select a Month:", style={"font-size": "18px", 
                                                 'margin-left': '35px', 
                                                 'margin-top': '20px'}),
            dcc.Dropdown(id="months_drop",
                         options=[
                             {"label": "January", "value": 0},
                             {"label": "February", "value": 1},
                             {"label": "March", "value": 2},
                             {"label": "April", "value": 3},
                             {"label": "May", "value": 4},
                             {"label": "June", "value": 5},
                             {"label": "July", "value": 6},
                             {"label": "August", "value": 7},
                             {"label": "September", "value": 8},
                             {"label": "October", "value": 9},
                             {"label": "November", "value": 10},
                             {"label": "December", "value": 11}],
                         multi=False,
                         value=0,
                         style={'width': "90%",
                                'color':'black',
                                'margin-left': '20px', 
                                'margin-top': '18px',}
                         ),
            html.Label("Color Highlight:", style={"font-size": "18px",
                                                         'margin-left': '40px',
                                                         'margin-top': '30px'}),
            dcc.Dropdown(id = "color_drop",
                         options = [
                             {"label": "Potential Fishing Zone (PFZ SCORE)", "value" : 1},
                             {"label": "Potential Fishing Zone (PFZ CATEGORY)", "value" : 2},
                             {"label": "Sea Surface Temperature (SST)", "value" : 3},
                             {"label": "Sea Surface Chlorophyll-A Content (SSCC)", "value" : 4},
                             {"label": "THERMAL FRONT", "value" : 5}],
                         multi = False,
                         value = 2,
                         style={'width': "90%",
                                'color':'black',
                                'margin-left': '20px', 
                                'margin-top': '15px'}
                        ),
            html.Div(
                dbc.Button('Generate', id='generate_button', n_clicks=0, color='info'),
                style={'text-align': 'center', 'margin-top': '15px'}
            ),
            html.Div([   
                html.H4("Sector Scope: ", style={'margin-top': '23px'}),
                html.P(["Sector 1: Don Marcelino", html.Br(),
                      "Sector 2: Don Marcelino(Outside Municipal Waters)", html.Br(),
                      "Sector 3: Malita", html.Br(),
                      "Sector 4: Governor Generoso", html.Br(),
                      "Sector 5: Sta. Maria, Sulop, Padada, Hagonoy, Digos", html.Br(),
                      "Sector 6: Outside Municipal Waters", html.Br(),
                      "Sector 7: Governor Generoso", html.Br(),
                      "Sector 8: Sta. Cruz, Davao City", html.Br(),
                      "Sector 9: San Isidro, Lupon, Banaybanay, Samal (Southern Region)", html.Br(),
                      "Sector 10: Davao City, Panabo, Tagum, Samal (Northern Region), Maco, Mabini, Pantukan"]
                      )
                ], style={'margin-left': '40px', 'max-width': '300px'})
                
        ], width=3, style={'outline': '1px solid white'}),
        dbc.Col([    
            dcc.Loading(
                html.Div(id='map-container', children=[
                    dcc.Store(id='store_data'),
                    dbc.Row([   
                        dbc.Col([
                            dcc.Graph(id='pfz_map')
                        ], width=12)
                    ]),
                    dbc.Row([
                        dbc.Col([
                            dcc.Graph(id='pfz_pie'),
                            html.Label("Select a Sector:", style={"font-size": "18px"}),
                            html.Div([
                                dcc.Slider(
                                    id="index-slider",
                                    min=min(sector_points_dict.keys()),
                                    max=max(sector_points_dict.keys()) + 1,
                                    step=1,
                                    value=None,
                                    marks={0: "1", 1: "2", 2: "3", 3: "4", 4: "5", 5: "6", 6: "7", 7: "8", 8: "9", 9: "10", 10: "Total"}, 
                                )
                            ], style={"font-weight": "bold"})  # Add this line to make the slider text bold
                        ], width=6),
                        dbc.Col([
                            dcc.Graph(id='area_bar')
                        ], width=6)
                    ]),

                ]),
                id="loading-2",
                type="graph"
            )
        ], width=9)
    ])
], fluid=True)


In [16]:
mant = None

In [17]:
get = None

In [18]:
# store callback test
@app.callback(
    [Output('store_data', 'data'),
     Output('pfz_map', 'figure'),
     Output('area_bar', 'figure'),
     Output('index-slider', 'value')],
    [Input('generate_button', 'n_clicks')],
    [State('months_drop', 'value'),
     State('color_drop', 'value')]
)
def update_data_and_graphs(n_clicks, month, color):
    
    # Generate and return data to be stored
    my_data = {}
    stored_data = my_data
    
    colors = {
    'background': '#3e3d40',
    'text': 'white'
    }

    # Update graphs
    global fig
    global cluster_gdf
    global cluster_group_area
    global all_gdf
    global get
    
    if (month is None or month < 0) or (color is None or color < 0): 
        # Raises a PreventUpdate exception if month or color is not specified or is less than zero
        raise PreventUpdate
    else:

        bar_colors = ['steelblue', 'greenyellow', 'midnightblue'] # List of colors for each variable in the stacked bar
        month_df = pd.read_csv(file_paths[month]) # Reads in a CSV file specified by the "month" variable
        mdf = month_df
        mdf['PFZ_score'], mdf['pfz'] = MCDS(month_df) # Applies a function "MCDS" to the "month_df" DataFrame and adding two new columns: "PFZ_score" and "pfz"

        month_gdf = gpd.GeoDataFrame(
            mdf, 
            geometry=gpd.points_from_xy(mdf.lon, mdf.lat)
        )
        
        get = mdf

        for key, sector_points in sector_points_dict.items():
            merged_df = pd.merge(sector_points_dict[key], month_gdf, on=['lat', 'lon'], how='inner', suffixes = ('_old', ''))
            merged_df = merged_df.filter(regex='^(?!.*_old)')
            sector_points_dict[key] = merged_df

        #update all_gdf
        update = pd.merge(all_gdf, month_gdf, on=['lat', 'lon'], how='inner', suffixes = ('_old', ''))
        update = update.filter(regex='^(?!.*_old)')
        all_gdf = update

        map_figure(mdf, color)
        global mant
        mant = mdf
        new_cga()

        
         #BAR PLOT
        if all_gdf['pfz'].str.contains('Low').any():
            cluster_trace("Low")
        if all_gdf['pfz'].str.contains('Moderate').any():
            cluster_trace("Moderate")
        if all_gdf['pfz'].str.contains('High').any():
            cluster_trace("High")

       # Creates a stacked bar chart showing fishing zone area for different sectors
        bar_fig = px.bar(cluster_group_area,
                         x=cluster_group_area.index + 1,  # Add 1 to the index values
                         y=['Moderate_area_km2', 'High_area_km2', 'Low_area_km2'],
                         barmode='stack', title="Fishing Zone Area in Square Kilometers",
                         color_discrete_sequence=bar_colors)

        # Update the x-axis labels
        bar_fig.update_layout(
            xaxis=dict(
                tickmode='linear',
                tick0=1,  # Start the ticks at 1
                dtick=1  # Set the tick interval to 1
            )
        )
        
        # Update the chart layout and formatting settings, including axis titles, background colors, and font styles
        bar_fig.update_layout(
            xaxis_title=dict(
                text="SECTORS",
                font=dict(color=colors['text'])
            ),
            yaxis_title=dict(
                text="area in km2",
                font=dict(color=colors['text'])
            ),
            plot_bgcolor=colors['background'],
            paper_bgcolor=colors['background'],
            font=dict(color=colors['text'], size=14)
        )
        
        # Update the traces of the chart (i.e., the bars)
        bar_fig.update_traces(
            marker=dict(
                  line=dict(color='#ffffff', width=1)
            ),
            textfont=dict(
                color=colors['text'],
                size=14
            )
        )
        
        # Update the x-axis and y-axis labels
        bar_fig.update_layout(xaxis_title="SECTORS", yaxis_title="AREA (KM²)")


        return stored_data, fig, bar_fig, 10


In [19]:
@app.callback(
    Output('pfz_pie', 'figure'),
    Input('index-slider', 'value')
)
def update_pfz_pie(value):
    global cluster_gdf
    global sector_points_dict

    if value == 10:
        # Create a GeoDataFrame for all sectors
        gdf = pd.concat(sector_points_dict.values())
        title = ""
    else:
        # Get the GeoDataFrame for this index
        gdf = sector_points_dict.get(value)  # Use .get() method to handle missing keys 
        if gdf is not None:
            sector_number = value + 1  # Add 1 to the value to get the sector number
            title = f"Sector {sector_number}"
        else:
            title = "All Sectors"

    pfz_colors = {'Low': 'midnightblue', 
                  'Moderate': 'steelblue', 
                  'High': 'greenyellow'}
    # Count the number of data points in each category
    category_counts = gdf['pfz'].value_counts()

    # Create a pie chart
    data = [go.Pie(labels=category_counts.index, values=category_counts.values,
                   marker=dict(colors=[pfz_colors[i] for i in category_counts.index]))]
    layout = go.Layout(title=f'Percentage Distribution of Fishing Zones {title}')
    pie_fig = go.Figure(data=data, layout=layout)    
    
    # Update the pie chart traces
    pie_fig.update_traces(
        # Set the colors of the chart
        marker=dict(
            # Set the color and width of the outline
            line=dict(color='white', width=1)
        ),
        # Set the font color and size for the text
        textfont=dict(
            color='white',
            size=14
        ),
        # Set the information that appears when hovering over the chart
        hovertemplate='PFZ: %{label}<br>Count: %{value}<br>Percentage: %{percent}',
        # Set the size of the hole in the center of the chart
        hole=0.5
        # Set the text that appears beside the legend
    )

    # Update the layout of the chart
    pie_fig.update_layout(
        # Set the background color of the chart
        plot_bgcolor='#3e3d40',
        # Set the background color of the paper
        paper_bgcolor='#3e3d40',
        # Set the font color and size for the chart title and axis labels
        font=dict(color='white', size=14)
    )  
    
    return pie_fig


In [20]:
lat_min = 5.61
lat_max = 7.73
lon_min = 125.16
lon_max = 126.56

In [21]:
# from mpl_toolkits.basemap import Basemap
import matplotlib.colors as colors

In [22]:
# for file in file_paths:
#     month_df = pd.read_csv(file)
    
#     month_df['PFZ_score'], month_df['pfz'] = MCDS(month_df)
    
#     data = month_df

#     fig = plt.figure(figsize=(8, 8))

#     # Define the colors for the colormap
#     colors_list = [ 'greenyellow', 'steelblue', 'midnightblue']

#     # Create the colormap
#     cmap = colors.ListedColormap(colors_list)

#     map = Basemap(projection='merc',llcrnrlon= lon_min, llcrnrlat= lat_min, urcrnrlon= lon_max, urcrnrlat= lat_max, resolution='i')

#     x, y = map(data['lon'].values, data['lat'].values)

#     map.scatter(x, y, c=data['PFZ_score'], cmap= plt.cm.viridis, vmin = 1.3, vmax = 2.6, edgecolors='none', alpha=0.75)

#     map.drawcountries()

#     map.colorbar(location='bottom', pad='10%')

#     # Add latitudes and longitudes
#     map.drawparallels(range(int(lat_min), int(lat_max)+1), labels=[1,0,0,0])
#     map.drawmeridians(range(int(lon_min), int(lon_max)+1), labels=[0,0,0,1])

#     # Set land color to gray
#     map.fillcontinents(color='gray', lake_color='none')

#     plt.show()
#     plt.close(fig)

In [24]:
if __name__ == '__main__':
    app.run_server(debug = True, port = 8053)

Dash is running on http://127.0.0.1:8053/

Dash app running on http://127.0.0.1:8053/


In [ ]:
cluster_group_area

In [ ]:
all_gdf

In [ ]:
cluster_group_area

In [ ]:
mant